In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
# !git clone https://github.com/CarperAI/trlx.git
%cd drive/MyDrive/ClauseSummary/trlx
# pip install torch==2.0.0 --extra-index-url https://download.pytorch.org/whl/cu116 # for cuda
# pip install -e .

/content/drive/MyDrive/ClauseSummary/trlx


In [8]:
!pip install transformers
!pip install datasets
!pip install tqdm
!pip install torchtyping

  Using cached datasets-2.13.1-py3-none-any.whl (486 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached xxhash-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)


In [9]:
import os
from typing import List

import torch
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoTokenizer

### Train

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import trlx

tokenizer = AutoTokenizer.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-summarization")

# Load dataset
# train_dataset, val_dataset = ###-dataset

ModuleNotFoundError: ignored

### Reward Model import

### PPO

In [ ]:
from trlx.data.default_configs import (
    ModelConfig,
    OptimizerConfig,
    PPOConfig,
    SchedulerConfig,
    TokenizerConfig,
    TrainConfig,
    TRLConfig,
)

default_config = TRLConfig(
    train=TrainConfig(
        seq_length=128,
        epochs=100,
        total_steps=100000,
        batch_size=12,
        checkpoint_interval=10000,
        eval_interval=100,
        pipeline="PromptPipeline",
        trainer="AcceleratePPOTrainer",
        save_best=False,
    ),
    model=ModelConfig(
        model_path="KETI-AIR-Downstream/long-ke-t5-base-summarization",
        num_layers_unfrozen=-1,
        model_arch_type="seq2seq",
    ),
    tokenizer=TokenizerConfig(
        tokenizer_path="KETI-AIR-Downstream/long-ke-t5-base-summarization",
        padding_side="right",
        truncation_side="right",
    ),
    optimizer=OptimizerConfig(
        name="adamw",
        kwargs={
            "lr": 5.0e-5,
            "betas": [0.9, 0.999],
            "eps": 1.0e-8,
            "weight_decay": 1.0e-6,
        },
    ),
    scheduler=SchedulerConfig(
        name="cosine_annealing",
        kwargs={
            "T_max": 100000,
            "eta_min": 5.0e-5,
        },
    ),
    method=PPOConfig(
        name="PPOConfig",
        num_rollouts=128,
        chunk_size=12,
        ppo_epochs=4,
        init_kl_coef=0.05,
        target=6,
        horizon=10000,
        gamma=0.99,
        lam=0.95,
        cliprange=0.2,
        cliprange_value=0.2,
        vf_coef=1,
        scale_reward=None,
        ref_mean=None,
        ref_std=None,
        cliprange_reward=10,
        gen_kwargs={
            "max_new_tokens": 50,
            "do_sample": True,
            "top_k": 0,
            "top_p": 1,
            "eos_token_id": -1,
        },
    ),
)

In [ ]:
 config = TRLConfig.update(default_config, hparams={})

In [ ]:
    trlx.train(
        prompts=prompts,
        eval_prompts=val_prompts,
        reward_fn=metric_fn,
        config=config,
    )